In [1]:
#import pandas as pd
from pandas import Series
import numpy as np
from joblib import Parallel, delayed
import json
from scipy.sparse import hstack
import time
from datetime import datetime
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.preprocessing.data import OneHotEncoder
from sklearn import preprocessing


Using TensorFlow backend.
/Users/wjj/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/wjj/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/wjj/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/wjj/anaconda3/lib/python3.7/site-packages/tensorflow/py

# 工具函数

In [2]:
#年龄处理
time_now = int(time.time())
time_local = time.localtime(time_now)
now_year = time.strftime("%Y-%m-%d %H:%M:%S",time_local)
now_year = int(now_year[:4])
def age_process(birthday):
    #len(list(user_df.loc[2,'birthday'])) = 12,后三位为毫秒
    try:
        time_local = time.localtime(int(birthday[:9]))
    except:
        return 4
    birth_year = time.strftime("%Y-%m-%d %H:%M:%S",time_local)
    age = now_year - int(birth_year[:4])
    if age==0:
        return 0
    elif age<18:
        return 1
    elif age<20:
        return 3
    elif age<22:
        return 4
    elif age<24:
        return 5
    elif age<26:
        return 6
    elif age<28:
        return 7
    elif age<30:
        return 8
    elif age<32:
        return 9
    elif age<34:
        return 10
    elif age<36:
        return 11
    elif age<38:
        return 12
    elif age<45:
        return 13
    elif age<60:
        return 14
    else:
        return 15

def register_type_fun(row):
    register_type_list = row.split(',')
    tmp_list = [0,0,0,0,0,0]
    if 'weixin' in register_type_list:
        tmp_list[0] = 1
    if 'sns' in register_type_list:
        tmp_list[1] = 1
    if 'sina_weibo' in register_type_list:
        tmp_list[2] = 1
    if 'qq_zone' in register_type_list:
        tmp_list[3] = 1
    if 'missing' in register_type_list:
        tmp_list[5] = 1
    if 'cellphone' in register_type_list:
        tmp_list[4] = 1
    if 'xiaomi' in register_type_list:
        tmp_list[5] = 1
    if 'device' in register_type_list:
        tmp_list[5] = 1
    return tmp_list

def columbus_user_tags_fun(row):
    columbus_user_tags_list = row.split(',')
    tmp_list = [0,0,0,0,0,0]
    if 'weixin' in register_type_list:
        tmp_list[0] = 1
    if 'sns' in register_type_list:
        tmp_list[1] = 1
    if 'sina_weibo' in register_type_list:
        tmp_list[2] = 1
    if 'qq_zone' in register_type_list:
        tmp_list[3] = 1
    if 'missing' in register_type_list:
        tmp_list[5] = 1
    if 'cellphone' in register_type_list:
        tmp_list[4] = 1
    if 'xiaomi' in register_type_list:
        tmp_list[5] = 1
    if 'device' in register_type_list:
        tmp_list[5] = 1
    return tmp_list

def time_process(data_time):
    try:
        time_tmp = time.localtime(int(data_time[:9]))
    except:
        return 4
    time_tmp = time.strftime("%Y-%m-%d %H:%M:%S",time_tmp)
    year = list(time_tmp)[:4]
    month = list(time_tmp)[5:7]
    day = list(time_tmp)[9:11]
    hour = list(time_tmp)[3]
    return year,month,day,hour 

def activate_channel_label_fun(slug):
    slug = str(slug)
    if slug=='record_eating_activity':
        return 1
    elif slug=='record_weight':
        return 2
    elif slug=='post_create':
        return 3
    elif slug=='habit_checkin':
        return 4
    elif slug=='search_food':
        return 5
    elif slug=='finished_sports':
        return 6
    else:  
        return 0
    
def special_condition_label_fun(x):
    x = str(x)
    if x=='0.0' or x=='0':
        return 0
    elif x=='1.0' or x=='1':
        return 1
    elif x=='2.0' or x=='2':
        return 2
    elif x=='3.0' or x=='3':
        return 3
    else:  
        return 4

# 四分位箱线图处理异常值
def box_plot(data_series,feature_name):
    data_series = pd.Series(sorted(data_series))
    #q_abnormal_low = data_series.quantile(0.25) - 1.5 * (data_series.quantile(0.75) - data_series.quantile(0.25))
    q_abnormal_up = data_series.quantile(0.90) + 1.5 * (data_series.quantile(0.90) - data_series.quantile(0.25))
    q_abnormal_up = int(round(q_abnormal_up,0))
    data_series = data_series.apply(lambda x:q_abnormal_up if x>q_abnormal_up else x)
    return data_series
    
def _load_data(data_file):
    user_df = pd.read_csv(filepath_or_buffer = data_file,sep=",",names=user_feature_list)
    user_df.drop(labels=None,axis=0, index=0, columns=None, inplace=True)
    return user_df

def app_version_func(app_version):
    if app_version == '7.4' or app_version =='9.4':
        app_version = 'high'
    elif app_version == '7.3' or app_version =='9.3':
        app_version = 'medium'
    elif app_version == 'nan':
         app_version == 'missing'
    else:
        app_version = 'low'
    return app_version


# 时间函数

In [3]:
def timestamp_to_year(date_timestamp): 
    time_tmp = time.localtime(int(date_timestamp/1000))
    date = time.strftime("%Y-%m-%d %H:%M:%S",time_tmp)
    year = int(date[:4]) 
    
    if year <= 2017:
        year=2017
    return year

def timestamp_to_month(date_timestamp): 
    time_tmp = time.localtime(int(date_timestamp/1000))
    date = time.strftime("%Y-%m-%d %H:%M:%S",time_tmp) 
    month = int(date[5:7])
    return month

def timestamp_to_day(date_timestamp): 
    time_tmp = time.localtime(int(date_timestamp/1000))
    date = time.strftime("%Y-%m-%d %H:%M:%S",time_tmp)
    day = int(date[8:11])
    return day

def timestamp_to_hour(date_timestamp): 
    time_tmp = time.localtime(int(date_timestamp/1000))
    date = time.strftime("%Y-%m-%d %H:%M:%S",time_tmp)
    hour = int(date[11:13])
    if 12 < hour <= 7:
        hour = 'hour_night'
    elif 8 < hour <= 11:
        hour = 'hour_moring'
    elif 11 < hour <= 13:
        hour = 'hour_noon'
    elif 13 < hour <= 17:
        hour = 'hour_afternoon'
    elif 18 < hour <= 12:
       hour = 'hour_evening'
    else:
        hour = 'hour_missing'
    return hour

def timestamp_to_week(date_timestamp): 
    date_datetime = datetime.fromtimestamp(int(date_timestamp/1000)) 
    week = date_datetime.strftime('%A')
    if week == 'Sunday' or week == 'Saturday':
        week == 'weekend'
    elif week == 'Tuesday':
        week == 'tuesday'
    elif week == 'Monday' or week == 'Wednesday' or week == 'Thursday' or week == 'Friday':
        week == 'weekday'
    else:
        week =="week_missing"
    return week

def timestamp_to_season(date_timestamp): 
    time_tmp = time.localtime(int(date_timestamp/1000))
    date = time.strftime("%Y-%m-%d %H:%M:%S",time_tmp) 
    month = int(date[5:7])
    if month == 3 or month == 4 or month == 5:
        season = 'season_spring'
    elif month == 6 or month == 7 or month == 8:
        season = 'season_summer'
    elif month == 9 or month == 10 or month == 11:
        season = 'season_autumn'
    elif month == 12 or month == 1 or month == 2:
        season = 'season_winter'
    else:
        season = 'season_missing'
    return season


# slug转化函数

In [133]:
def slug_to_recommendation_pool(user_df):
    user_df.loc[user_df['slug']=='p_ef_dzdqjrb_80g',"slug"] = 's_ef_dzdqjrb_80g'
    user_df.loc[user_df['slug']=='ef_dzdqjrb_15_20191206',"slug"] = 's_ef_dzdqjrb_80g'
    user_df.loc[user_df['slug']=='s_easyfun_yingzuidou_180g',"slug"] = 's_easyfun_yingzuidou_180g'
    user_df.loc[user_df['slug']=='p_ef_dzhjjrc_300g',"slug"] = 's_ef_dzhjjrc_300g'
    user_df.loc[user_df['slug']=='p_ef_jrc_yw_hj_2dai',"slug"] = 's_ef_dzhjjrc_300g'
    user_df.loc[user_df['slug']=='p_ef_jrc_xl_hj_2dai',"slug"] = 's_ef_dzhjjrc_300g'
    user_df.loc[user_df['slug']=='p_ef_dzhjjrc_600g',"slug"] = 's_ef_dzhjjrc_300g'
    #本来就在推荐池内的
    recommend_pool = pd.read_excel('/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/good_feature_data/recommend_pool.xlsx')
    recommend_pool_list = recommend_pool['slug'].tolist()
    user_pool_df = user_df[user_df['slug'].isin(recommend_pool_list)]
    user_pool_df = user_pool_df.rename(columns={'slug':'s_slug'})
    #在推荐池中的P开头的SKU，下一步匹配游有用
    goods_match_on_sale_df = pd.read_excel('./good_feature_data/goods_match_on_sale_s.xls')
    goods_match_on_sale_df = goods_match_on_sale_df[['s_slug','p_slug','goods_slug']]
    goods_match_on_sale_df[['p_slug','goods_slug']] = goods_match_on_sale_df[['p_slug','goods_slug']].applymap((lambda x: "".join(x.split()) if type(x) is str else x))
    goods_match_on_sale_df['goods_slug'] = goods_match_on_sale_df['goods_slug'].astype(str)
    goods_match_on_sale_df['p_slug'] = goods_match_on_sale_df['p_slug'].astype(str)
    goods_match_on_sale_df['s_slug'] = goods_match_on_sale_df['s_slug'].astype(str)
    user_df['slug'] = user_df['slug'].astype(str)
    p_list = ['p_ea10days_1512g','p_easyace_ysjnew_xmy_190828','p_ef_funsichi_nwlb','p_ef_zkxsd_757g','p_ea_3daypttp'
    ,'p_7394376616792','p_7394376616815','p_easyfun_madai_tea_20']
    pp_user_df = user_df[user_df['slug'].isin(p_list)]
    #pp_not_buy_match_df = pp_not_buy_match_df.rename(columns={'slug': 's_slug'})
    #pp_user_df = pp_user_df.drop_duplicates('slug')
    pp_user_df = pp_user_df.groupby(by=['slug'])['c'].sum().to_frame()
    pp_user_df['slug'] =pp_user_df.index
    pp_user_df =pp_user_df.reset_index(drop= True)
    #匹配
    pp_user_df = pd.merge(pp_user_df,goods_match_on_sale_df,right_on='p_slug',left_on='slug',how='inner')
    goods_user_match_df = pd.merge(user_df,goods_match_on_sale_df,right_on='goods_slug',left_on='slug',how='inner')
    #找到匹配成功的
    goods_user_match_df = goods_user_match_df[~goods_user_match_df['goods_slug'].isnull()]
    pp_user_df = pp_user_df[~pp_user_df['p_slug'].isnull()]
    #goods_user_match_df = goods_user_match_df.groupby(by=['s_slug'])['c'].sum()
    #goods_user_match_df = goods_user_match_df.drop_duplicates('s_slug')
    #pp_user_df = pp_user_df.groupby(by=['s_slug'])['c'].sum()
    user_match_df =pd.concat([goods_user_match_df,pp_user_df],sort=False)
    user_match_df = user_match_df.groupby(by=['s_slug'])['c'].sum().to_frame()
    user_match_df['s_slug'] =user_match_df.index
    user_match_df =user_match_df.reset_index(drop= True)
    user_match_pool_df = user_match_df[user_match_df['s_slug'].isin(recommend_pool_list)]
    user_all_df = pd.concat([user_match_pool_df,user_pool_df],sort=False)
    user_all_df = user_all_df.groupby(by=['s_slug'])['c'].sum().to_frame()
    user_all_df['s_slug'] =user_all_df.index
    user_all_df =user_all_df.reset_index(drop= True)
    user_all_df = user_all_df[['s_slug','c']]
    return user_all_df

In [4]:
data_file_list = [
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_12.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_11.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_10.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_9.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_8.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_7.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_6.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_5.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_4.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_3.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_2.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_1.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_0.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_-12.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_-11.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_-10.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_-9.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_-8.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_-7.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_-6.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_-5.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_-4.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_-3.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_-2.csv',
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_-1.csv',
]
user_feature_list = [
 'second_id','slug','time','birthday','register_timestamp','first_visit_time','survey_finished_at',
 'first_traffic_source_type','register_source_app_key','register_type',
 'register_sns','gender','phone_model','app_channel','app_version','os_version','province','city_level','purpose',
 'target_weight','start_weight','latest_weight','bmi','budget_calory',
 'view_goods_count','add_cart_count','commit_order_count','num_paid','eka_num_paid',
 'total_cost','recent_buy_at','record_food_count','search_food_count','record_weight_count',
 'finished_sports_count','habbit_checkin_count','post_create_count',
 'active_day_count','activate_channel','special_condition','easy_customer_first_buy_time',
 'easy_membership_first_platform','easy_membership_first_time','user_membership_status','user_fans_status',
 'guide_to_bh_at','wx_assistant_friend','columbus_user_tags'
]


In [5]:
data_file_list = [
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_12.csv']
user_feature_list = [
 'second_id','slug','time','birthday','register_timestamp','first_visit_time','survey_finished_at',
 'first_traffic_source_type','register_source_app_key','register_type',
 'register_sns','gender','phone_model','app_channel','app_version','os_version','province','city_level','purpose',
 'target_weight','start_weight','latest_weight','bmi','budget_calory',
 'view_goods_count','add_cart_count','commit_order_count','num_paid','eka_num_paid',
 'total_cost','recent_buy_at','record_food_count','search_food_count','record_weight_count',
 'finished_sports_count','habbit_checkin_count','post_create_count',
 'active_day_count','activate_channel','special_condition','easy_customer_first_buy_time',
 'easy_membership_first_platform','easy_membership_first_time','user_membership_status','user_fans_status',
 'guide_to_bh_at','wx_assistant_friend','columbus_user_tags'
]


In [18]:
import pandas as pd  
user_df = pd.DataFrame(columns = user_feature_list)
for i in data_file_list:
    user_tmp_df =  _load_data(i)
    user_df = pd.concat([user_df,user_tmp_df],ignore_index=True)
user_df = user_df[~user_df['second_id'].isin(['second_id'])]
user_df = user_df.drop_duplicates('second_id')
del user_df['columbus_user_tags']
del user_df['os_version']
user_df = user_df.head(100000)

In [19]:
#phonemodel特征,不要重复跑，不然报错
#user_df['phone_model'] = user_df['phone_model'].fillna('1')
user_df.loc[(user_df["phone_model"]=='iPhone'),"phone_model"] = 2
user_df.loc[(user_df["phone_model"]!='iPhone'),"phone_model"] = 1

In [20]:
#purpose
user_df.loc[(user_df["purpose"]=='lost_weight'),"purpose_label"] = 1
user_df.loc[(user_df["purpose"]=='keep_muscle'),"purpose_label"] = 2
user_df.loc[(user_df["purpose"]=='gain_muscle'),"purpose_label"] = 3
user_df.loc[(user_df["purpose"]!='lost_weight') & (user_df["purpose"]!='keep_muscle') &(user_df["purpose"]!='gain_muscle'),"purpose_label"] = 4
user_df['purpose_label'] = user_df['purpose_label'].fillna(5)
del user_df['purpose']

In [21]:
#个人信息特征空值，不要重复跑，不然报错
user_df['city_level'].fillna(2,inplace=True)
user_df['city_level'] = user_df['city_level'].astype(np.int32)
#province_list = user_df['province'].value_counts().keys().tolist()[:37]
province_list=['广东省', '江苏省', '浙江省', '北京', '上海', '山东省', '河南省', '四川省', '湖北省', '河北省', '湖南省', '安徽省', '辽宁省', '福建省', '陕西省', '重庆', '江西省', '天津', '云南省', '黑龙江省', '山西省', '未知', '广西', '吉林省', '内蒙古', '贵州省', '新疆', '甘肃省', '海南省', '宁夏', '香港', '青海省', '台湾省', '澳门', '西藏']
user_df['province'].fillna('未知',inplace = True)
user_df['province'] = user_df['province'].apply(lambda x:x if x in province_list else '国外')
encoder = preprocessing.LabelEncoder()
encoder.fit(user_df['province'])
user_df['province'] = encoder.transform(user_df['province'])
user_df["gender"].fillna(2, inplace=True)
user_df['gender'] = user_df['gender'].apply(lambda x:1 if x=='1' or x==1 or x==1.0 else 2)
user_df["activate_channel"] = user_df["activate_channel"].apply(activate_channel_label_fun)
user_df["special_condition"] = user_df["special_condition"].apply(special_condition_label_fun)

In [22]:
#用户行为特征
feature_box_plot_list = ['view_goods_count', 'record_food_count', 'search_food_count','record_weight_count','active_day_count']                              
for i in feature_box_plot_list:
    user_df[i].fillna(0,inplace=True)
    user_df[i] = user_df[i].astype(np.int32)
    user_df[i] = box_plot(user_df[i],i)
    user_df[i].fillna(0,inplace=True)
    
# PND模型处理异常值
feature_custom_list = ['habbit_checkin_count','total_cost','num_paid','add_cart_count','eka_num_paid','commit_order_count','post_create_count','finished_sports_count']
for i in feature_custom_list:
    user_df[i].fillna(0,inplace=True)
    user_df[i] = user_df[i].astype(np.float32)
user_df['habbit_checkin_count'] = user_df['habbit_checkin_count'].apply(lambda x:200 if x>200 else x)
user_df['add_cart_count'] = user_df['add_cart_count'].apply(lambda x:80 if x>80 else x)
user_df['total_cost'] = user_df['total_cost'].apply(lambda x:5000 if x>5000 else x)
user_df['num_paid'] = user_df['num_paid'].apply(lambda x:20 if x>20 else x)
user_df['eka_num_paid'] = user_df['eka_num_paid'].apply(lambda x:20 if x>20 else x)
user_df['commit_order_count'] = user_df['commit_order_count'].apply(lambda x:30 if x>30 else x)
user_df['post_create_count'] = user_df['post_create_count'].apply(lambda x:50 if x>50 else x)
user_df['finished_sports_count'] = user_df['finished_sports_count'].apply(lambda x:80 if x>80 else x)
   

In [23]:
#用户体制特征使用平均填充
feature_body_list = ["target_weight","start_weight","latest_weight","bmi","budget_calory"]
for i in feature_body_list:
    user_df[i] = user_df[i].astype(np.float32)
    mean_val = user_df[i].mean()
    user_df[i].fillna(mean_val, inplace=True)
del mean_val

In [24]:
#时间特征
#年龄特征处理
user_df['birthday'] = user_df['birthday'].astype(str)
user_df['birthday'] = user_df['birthday'].apply(age_process)

In [25]:
user_df['register_sns'] = user_df['register_sns'].fillna('missing')
user_df= user_df.join(pd.get_dummies(user_df['register_sns'],prefix='register_sns_'))
del user_df['register_sns']

In [26]:
#app_channel onehot
app_channel_list = ['missing', 'huawei','nearme', 'vivo','xiaomi','boohee','tencent','meizu', '_360','baidu','wandoujia','undefind','lenovo','zhihuan','oppo','other',]
user_df['app_channel'] = user_df['app_channel'].apply(lambda x:'vivo' if x == 'oppo' else x)
user_df['app_channel'] = user_df['app_channel'].apply(lambda x:x if x in app_channel_list else 'other')
user_df= user_df.join(pd.get_dummies(user_df['app_channel'],prefix='app_channel_'))
del user_df['app_channel']

In [27]:
#注册方式onehot
user_df['register_type'] = user_df['register_type'].fillna('missing')
user_df['register_type']= pd.DataFrame(user_df['register_type'].apply(register_type_fun))
user_df['register_type_weixin'] = user_df['register_type'].apply(lambda x:x[0])
user_df['register_type_sns'] = user_df['register_type'].apply(lambda x:x[1])
user_df['register_type_sina_weibo'] = user_df['register_type'].apply(lambda x:x[2])
user_df['register_type_qq_zone'] = user_df['register_type'].apply(lambda x:x[3])
user_df['register_type_cellphone'] = user_df['register_type'].apply(lambda x:x[4])
user_df['register_type_missing'] = user_df['register_type'].apply(lambda x:x[5])
del user_df['register_type']

In [28]:
user_df['first_traffic_source_type'].fillna('missing',inplace=True)
user_df['first_traffic_source_type'] = user_df['first_traffic_source_type'].apply(lambda x:'direct' if x == '直接流量' else x)
user_df['first_traffic_source_type'] = user_df['first_traffic_source_type'].apply(lambda x:'indirect' if x == '引荐流量'  else x)
user_df= user_df.join(pd.get_dummies(user_df['first_traffic_source_type'],prefix='first_traffic_source_type_'))
del user_df['first_traffic_source_type']

In [29]:
user_df['register_source_app_key'] = user_df['register_source_app_key'].apply(lambda x:x if x == 'ifood' or x == 'boohee_weixin' or x == 'one' or x == 'food' or x == 'nice' else 'missing')            
user_df= user_df.join(pd.get_dummies(user_df['register_source_app_key'],prefix='register_source_app_key_'))
del user_df['register_source_app_key']

In [30]:
user_df['easy_membership_first_platform'].fillna('missing',inplace=True)
user_df= user_df.join(pd.get_dummies(user_df['easy_membership_first_platform'],prefix='easy_membership_first_platform_'))
del user_df['easy_membership_first_platform']

In [31]:
user_df['user_membership_status'].fillna('missing',inplace=True)
user_df['user_membership_status'] = user_df['user_membership_status'].apply(lambda x:'member_expire' if x == 'member_canceled' else x)
user_df['user_membership_status'] = user_df['user_membership_status'].apply(lambda x:'member_expire' if x == 'member_inactive' else x)
user_df['user_membership_status'] = user_df['user_membership_status'].apply(lambda x:'member_ongoing' if x == 'associate_member' else x)
user_df= user_df.join(pd.get_dummies(user_df['user_membership_status'],prefix='user_membership_status_'))
del user_df['user_membership_status']

In [32]:
user_df['user_fans_status'].fillna('missing',inplace=True)
user_df= user_df.join(pd.get_dummies(user_df['user_fans_status'],prefix='user_fans_status_'))
del user_df['user_fans_status']

In [33]:
user_df['wx_assistant_friend'].fillna('missing',inplace=True)
user_df['wx_assistant_friend'] = user_df['wx_assistant_friend'].apply(lambda x:'yes' if x == '是' else x)
user_df= user_df.join(pd.get_dummies(user_df['wx_assistant_friend'],prefix='wx_assistant_friend_'))
del user_df['wx_assistant_friend']

user_df['columbus_user_tags'].fillna('missing',inplace=True)
columbus_user_tags_list =[]
for row in user_df.itertuples():
    tmp_list = getattr(row, 'columbus_user_tags').split(',')
    columbus_user_tags_list = columbus_user_tags_list+tmp_list
print(len(set(columbus_user_tags_list)))
columbus_user_tags_set = set(columbus_user_tags_list)

In [34]:
#时间戳特征处理
timestamp_feature_list = ['register_timestamp','first_visit_time','survey_finished_at',
 'recent_buy_at','easy_customer_first_buy_time','easy_membership_first_time','guide_to_bh_at']
for i in timestamp_feature_list:
    user_null_df = user_df[user_df[i].isnull() == True]
    user_df = user_df[user_df[i].isnull() == False]
    user_df[i] = user_df[i].astype(int)
    user_df[i+'_year'] = user_df[i].apply(timestamp_to_year)
    user_df= user_df.join(pd.get_dummies(user_df[i+'_year'],prefix=i+'_year_'))
    del user_df[i+'_year']
    user_df[i+'_season'] = user_df[i].apply(timestamp_to_season)
    user_df= user_df.join(pd.get_dummies(user_df[i+'_season'],prefix=i+'_season_'))
    del user_df[i+'_season']
    user_df[i+'_month'] = user_df[i].apply(timestamp_to_month)
    user_df= user_df.join(pd.get_dummies(user_df[i+'_month'],prefix=i+'_month_'))
    del user_df[i+'_month']
    user_df[i+'_week'] = user_df[i].apply(timestamp_to_week)
    user_df= user_df.join(pd.get_dummies(user_df[i+'_week'],prefix=i+'_week_'))
    del user_df[i+'_week']
    #user_df[i+'_day'] = user_df[i].apply(timestamp_to_day)
    #user_df= user_df.join(pd.get_dummies(user_df[i+'_day'],prefix=i+'_day_'))
    #del user_df[i+'_day']
    user_df[i+'_hour'] = user_df[i].apply(timestamp_to_hour)
    user_df= user_df.join(pd.get_dummies(user_df[i+'_hour'],prefix=i+'_hour_'))
    del user_df[i+'_hour']
    del user_df[i]
    del user_null_df[i]
    user_df = pd.concat([user_df,user_null_df],sort=False)
user_df = user_df.sample(frac=1)


In [35]:
#app版本特征
user_df['app_version'] = user_df['app_version'].fillna('nan')
user_df['app_version'] = user_df['app_version'].astype(str)
user_df['app_version'] = user_df['app_version'].apply(lambda x:x[:3])
user_df['app_version'] = user_df['app_version'].apply(app_version_func)
user_df= user_df.join(pd.get_dummies(user_df['app_version'],prefix=i+'app_version_'))


In [36]:
user_df = user_df.fillna(0)

In [37]:
print(len(user_df.columns.tolist()))

252


# slug维度的点击、加购、购买比例，转化率，排名

### 样本sku处理成推荐池内

In [38]:
user_df.loc[user_df['slug']=='p_ef_dzdqjrb_80g',"slug"] = 's_ef_dzdqjrb_80g'
user_df.loc[user_df['slug']=='ef_dzdqjrb_15_20191206',"slug"] = 's_ef_dzdqjrb_80g'
user_df.loc[user_df['slug']=='s_easyfun_yingzuidou_180g',"slug"] = 's_easyfun_yingzuidou_180g'
user_df.loc[user_df['slug']=='p_ef_dzhjjrc_300g',"slug"] = 's_ef_dzhjjrc_300g'
user_df.loc[user_df['slug']=='p_ef_jrc_yw_hj_2dai',"slug"] = 's_ef_dzhjjrc_300g'
user_df.loc[user_df['slug']=='p_ef_jrc_xl_hj_2dai',"slug"] = 's_ef_dzhjjrc_300g'
user_df.loc[user_df['slug']=='p_ef_dzhjjrc_600g',"slug"] = 's_ef_dzhjjrc_300g'

In [39]:
#本来就在推荐池内的
recommend_pool = pd.read_excel('/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/good_feature_data/recommend_pool.xlsx')
recommend_pool_list = recommend_pool['slug'].tolist()
user_pool_df = user_df[user_df['slug'].isin(recommend_pool_list)]
user_pool_df = user_pool_df.rename(columns={'slug':'s_slug'})
user_pool_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71748 entries, 27337 to 2522
Columns: 252 entries, second_id to guide_to_bh_atapp_version__nan
dtypes: float32(5), float64(189), int32(1), int64(12), object(4), uint8(41)
memory usage: 117.2+ MB


In [40]:
#在推荐池中的P开头的SKU，下一步匹配游有用
goods_match_on_sale_df = pd.read_excel('./good_feature_data/goods_match_on_sale_s.xls')
goods_match_on_sale_df = goods_match_on_sale_df[['s_slug','p_slug','goods_slug']]
goods_match_on_sale_df[['p_slug','goods_slug']] = goods_match_on_sale_df[['p_slug','goods_slug']].applymap((lambda x: "".join(x.split()) if type(x) is str else x))
goods_match_on_sale_df['goods_slug'] = goods_match_on_sale_df['goods_slug'].astype(str)
goods_match_on_sale_df['p_slug'] = goods_match_on_sale_df['p_slug'].astype(str)
goods_match_on_sale_df['s_slug'] = goods_match_on_sale_df['s_slug'].astype(str)
user_df['slug'] = user_df['slug'].astype(str)
p_list = ['p_ea10days_1512g','p_easyace_ysjnew_xmy_190828','p_ef_funsichi_nwlb','p_ef_zkxsd_757g','p_ea_3daypttp'
,'p_7394376616792','p_7394376616815','p_easyfun_madai_tea_20']
pp_user_df = user_df[user_df['slug'].isin(p_list)]
#pp_not_buy_match_df = pp_not_buy_match_df.rename(columns={'slug': 's_slug'})
pp_user_df = pp_user_df.drop_duplicates(['second_id','slug'])

In [41]:
#匹配
pp_user_df = pd.merge(pp_user_df,goods_match_on_sale_df,right_on='p_slug',left_on='slug',how='inner')
goods_user_match_df = pd.merge(user_df,goods_match_on_sale_df,right_on='goods_slug',left_on='slug',how='inner')

In [42]:
#找到匹配成功的
goods_user_match_df = goods_user_match_df[~goods_user_match_df['goods_slug'].isnull()]
pp_user_df = pp_user_df[~pp_user_df['p_slug'].isnull()]
goods_user_match_df = goods_user_match_df.drop_duplicates(['second_id','s_slug'])
pp_user_df = pp_user_df.drop_duplicates(['second_id','s_slug'])
user_match_df =pd.concat([goods_user_match_df,pp_user_df])
user_match_df = user_match_df.drop_duplicates(['second_id','s_slug'])
user_match_pool_df = user_match_df[user_match_df['s_slug'].isin(recommend_pool_list)]

In [43]:
user_all_df = pd.concat([user_match_pool_df,user_pool_df],sort=False)

In [44]:
del user_all_df['slug']
del user_all_df['p_slug']
del user_all_df['goods_slug']

In [45]:
user_all_df['time'] = user_all_df['time'].astype(str)
user_all_df['time'] = user_all_df['time'].apply(lambda x:x[:10])
user_all_df = user_all_df.drop_duplicates(['second_id','s_slug','time'])
del user_all_df['time']
user_all_df = user_all_df.sample(frac=1)

In [52]:
user_all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73264 entries, 127630 to 76829
Columns: 251 entries, second_id to s_slug
dtypes: float32(5), float64(189), int32(1), int64(12), object(3), uint8(41)
memory usage: 119.1+ MB


### 商品维度的查看、加购、购买数据处理成推荐池内

In [178]:
buy_goods_count_by_slug_df = pd.read_csv(filepath_or_buffer = '/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0127/buy_goods_count_by_slug.csv',sep=",")
add_cart_count_by_slug_df = pd.read_csv(filepath_or_buffer = '/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0127/add_cart_count_by_slug.csv',sep=",")
view_goods_count_by_slug_df = pd.read_csv(filepath_or_buffer = '/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0127/view_goods_count_by_slug.csv',sep=",")


In [179]:
buy_goods_count_by_slug_df = slug_to_recommendation_pool(buy_goods_count_by_slug_df)
add_cart_count_by_slug_df = slug_to_recommendation_pool(add_cart_count_by_slug_df)
view_goods_count_by_slug_df = slug_to_recommendation_pool(view_goods_count_by_slug_df)

In [180]:
recommend_pool = pd.read_excel('/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/good_feature_data/recommend_pool.xlsx')


In [181]:
recommend_pool_list = list(recommend_pool['slug'])
view_goods_count_by_slug_list = view_goods_count_by_slug_df['s_slug'].to_list()
add_cart_count_by_slug_list = add_cart_count_by_slug_df['s_slug'].to_list()
buy_goods_count_by_slug_list = buy_goods_count_by_slug_df['s_slug'].to_list()

In [183]:
list(set(recommend_pool_list).difference(set(view_goods_count_by_slug_list)))

[]

In [147]:
#这两个商品套餐或者单品，加和统计他们的购买数量

# 草稿本